<a href="https://colab.research.google.com/github/NeerajMehta15/Deep-Learning-projects/blob/main/2_Twitter_Sentiment_analysis_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
#PyTorch Imports
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim

#Pandas and Nump
import pandas as pd
import numpy as np

#NLP Libraries
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter

#Machine Learning and Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer

#Deep Learning Libraries
import torch.nn as nn
import torch.optim as optim


In [121]:
# Download necessary NLTK resources (done once)
nltk.download('punkt')
nltk.download('stopwords')

# Define stopwords and stemmer globally to avoid redundancy
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [122]:
df = pd.read_csv('/content/twitter_training.csv')

# New Section

In [123]:
#Renaming columns
df.columns = ['ID','Place','Label','Tweet']
df.head()

,ID,Place,Label,Tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [124]:
#Convering this to Numerical data
df['Label'].unique()
map = {'Positive':3,'Negative':2,'Neutral':1,'Irrelevant':0}
df['Label'] = df['Label'].map(map)

In [125]:
#Deleting ID and Label columns
df = df.drop(['ID','Place'],axis=1)

In [126]:
df.head()

,Label,Tweet
0,3,I am coming to the borders and I will kill you...
1,3,im getting on borderlands and i will kill you ...
2,3,im coming on borderlands and i will murder you...
3,3,im getting on borderlands 2 and i will murder ...
4,3,im getting into borderlands and i can murder y...


In [127]:
# Define the TextDataset class
class TextDataset(Dataset):
    def __init__(self, text):
        self.text = text

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        return self.text[idx]

In [128]:
# # Preprocessing function for the tweets (no NLTK dependencies)
# def preprocessing_sentences(sentences):
#     stop_words = set([
#         "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves",
#         "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
#         "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are",
#         "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
#         "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about",
#         "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up",
#         "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when",
#         "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no",
#         "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don",
#         "should", "now", "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren", "couldn", "didn", "doesn", "hadn",
#         "hasn", "haven", "isn", "ma", "mightn", "mustn", "needn", "shan", "shouldn", "wasn", "weren", "won", "wouldn"
#     ])

#     stemmer = PorterStemmer()

#     processed_sentences = []
#     for sentence in sentences:
#         sentence = sentence.lower()
#         tokens = sentence.split()

#         # Remove stop words and apply stemming
#         tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]

#         processed_sentences.append(' '.join(tokens))  # Join tokens back into a sentence
#     return processed_sentences

In [129]:
# Function to encode the tweets
def embedding_tweets(df, max_len=50):
    df['Tweet'] = df['Tweet'].fillna('')

    # # Preprocess the tweets
    # processed_tweets = preprocessing_sentences(df['Tweet'].tolist())
    # df['Tweet'] = processed_tweets

    # Split tweets into words
    tweets = df['Tweet'].str.split()

    # Create vocabulary and mapping of words to indices
    vocab = {word for tweet in tweets for word in tweet}
    word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}

    # Calculate vocabulary size
    vocab_size = len(word_to_idx) + 1
    print("Vocabulary size:", vocab_size)

    # Convert tweets to sequences of indices
    df['tokenized'] = tweets.apply(lambda x: [word_to_idx[word] for word in x])

    # Padding (Ensure uniform sequence length)
    df['padded'] = df['tokenized'].apply(lambda x: x[:max_len] + [0] * (max_len - len(x)))

    return df, vocab_size

In [130]:
df, vocab_size = embedding_tweets(df)
dataset = TextDataset(df[['padded', 'Label']].values.tolist())

Vocabulary size: 82367


In [137]:
# Print the first 5 samples in the dataset
for i in range(5):
    padded, label = dataset[i]
    print(f"Sample {i+1}:")
    print(f"Padded: {padded}")
    print(f"Label: {label}")
    print("-" * 50)

Sample 1:
Padded: [68414, 21479, 50929, 4853, 28983, 25941, 22164, 68414, 14639, 31941, 39747, 18349, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label: 3
--------------------------------------------------
Sample 2:
Padded: [27249, 61192, 38939, 7855, 22164, 15204, 14639, 31941, 39747, 18349, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label: 3
--------------------------------------------------
Sample 3:
Padded: [27249, 50929, 38939, 7855, 22164, 15204, 14639, 27051, 39747, 18349, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label: 3
--------------------------------------------------
Sample 4:
Padded: [27249, 61192, 38939, 7855, 52000, 22164, 15204, 14639, 27051, 39747, 7564, 18349, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [131]:
# Defining the class
class SentimentAnalysisCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SentimentAnalysisCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(embed_dim, 4)

    def forward(self, text):
        embedded = self.embedding(text).permute(0, 2, 1)
        conved = self.conv(embedded)
        relu = self.relu(conved)
        pooled = torch.mean(relu, dim=2)
        return self.fc(pooled)


In [132]:
# Initialize the model
embed_dim = 10
model = SentimentAnalysisCNN(vocab_size, embed_dim)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Prepare data for sentiment analysis
train_loader = DataLoader(dataset, batch_size=50, shuffle=True)

In [133]:
print(train_loader)

In [134]:
# Training the model
for epoch in range(10):
    model.train()  # Set model to training mode
    running_loss = 0.0  # Variable to accumulate loss over the epoch

    for padded, label in train_loader:
        # Move data to the appropriate device (GPU or CPU)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Convert padded to tensor if it's not already a tensor
        padded = torch.tensor(padded).to(device)
        label = torch.tensor(label).to(device)

        # Forward pass
        output = model(padded)

        # Calculate loss
        loss = criterion(output, label)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss for the epoch
        running_loss += loss.item()

    # Print average loss for the epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")


TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
# Example for testing after training
# Assuming you have test data in the same format as training data
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for padded, label in test_loader:
        padded = torch.LongTensor(padded).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        label = torch.LongTensor(label).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        output = model(padded)
        _, predicted = torch.max(output, 1)
        print(f"Predicted: {predicted}, Actual: {label}")

Test function
